In [2]:
# Selenium, ChromeDriver 및 Chrome 설치
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium beautifulsoup4 pandas

# Colab에서 Chrome 설정
import os
os.environ["PATH"] += ":/usr/lib/chromium-browser/"

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,513 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,738 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports/universe amd

In [3]:
import time
import random
import csv
from selenium.webdriver import Remote, ChromeOptions
from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from google.colab import files  # Colab에서 파일을 다운로드할 수 있도록 함

IP차단 방지(무료 크레딧 10달러)
https://brdta.com/kimfl

- scraping Browser > get started

AUTH = 'user:pass'
해당 부분에 각자 제공받은 코드를 넣으시면 됩니다.

- 참고로 계속 코드를 돌리게 될 때 쿠팡에 차단 당하지는 않지만 더이상 코드를 돌리지 못하게 되니
신중하게 시도하시면 좋을 것 같습니다!(저는 막 이메일 3~4개 정도 썼어요.. 이제 더이상 쓸게 없습니닷)
한개로 계속 시도하려면 유료로 이용을 해야 합니다

또 저는 자꾸 별점이 가져와지지 않는데.. ㅜㅜ 일단 리뷰라도 가져와지니 올립니다
각자 다른 상품을 하면 좋을 것 같습니다.

- 참고한 영상은
https://www.youtube.com/watch?v=QtrBHY4_uLw

1시간 10분? 정도부터 보시면 될 것 같아요.



In [4]:

# Enter your credentials - the zone name and password
AUTH = 'brd-customer-hl_056f900f-zone-scraping_browser1:bbi7wx37mu68'

SBR_WEBDRIVER = f'https://{AUTH}@zproxy.lum-superproxy.io:9515'

# 제품명 고정
PRODUCT_NAME = "유닉스 세라믹 헤어 매직기"

def main():
    print('Connecting to Scraping Browser...')
    sbr_connection = ChromiumRemoteConnection(SBR_WEBDRIVER, 'goog', 'chrome')
    with Remote(sbr_connection, options=ChromeOptions()) as driver:
        print('Connected! Navigating...')
        driver.implicitly_wait(5)
        driver.get('https://www.coupang.com/vp/products/3154423?itemId=14424741&vendorItemId=3089081003&pickType=COU_PICK&q=%EC%9C%A0%EB%8B%89%EC%8A%A4+%EC%84%B8%EB%9D%BC%EB%AF%B9+%ED%97%A4%EC%96%B4+%EB%A7%A4%EC%A7%81%EA%B8%B0&itemsCount=36&searchId=58d5667e0bd8451d8d7d79cb94b7519f&rank=0&searchRank=0&isAddedCart=')  # URL 수정 가능
        time.sleep(random.uniform(2, 5))
        driver.find_element(By.NAME, "review").click()
        time.sleep(random.uniform(2, 6))

        # 페이지 버튼을 찾고, 페이지 수 출력
        review_btns = driver.find_elements(By.CSS_SELECTOR, ".js_reviewArticlePageBtn")
        print(f"상품평 총 {len(review_btns)}페이지")
        print()

        # CSV 파일 초기화
        with open('/content/reviews.csv', mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['제품명', '별점', '리뷰내용'])  # 헤더 추가

        # 리뷰 페이지를 순차적으로 크롤링
        page_num = 1  # 페이지 번호
        while True:
            reviews = driver.find_elements(By.CSS_SELECTOR, ".js_reviewArticleReviewList .sdp-review__article__list__review")

            # 리뷰 데이터를 CSV에 저장
            for review in reviews:
                try:
                    # 별점 추출
                    rating_element = WebDriverWait(review, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "sdp-review__article__list__info__product-info__star-orange.js_reviewArticleRatingValue"))
                    )
                    rating = rating_element.get_attribute("data-rating")
                except Exception as e:
                    print(f"별점 추출 실패: {e}")
                    rating = "없음"

                try:
                    # 리뷰 내용 추출
                    review_content = review.find_element(By.CSS_SELECTOR, ".sdp-review__article__list__review__content.js_reviewArticleContent").text
                except:
                    review_content = "없음"

                # 리뷰 내용이 없으면 크롤링 중지
                if review_content == "없음":
                    print("리뷰 내용이 없습니다. 크롤링을 종료합니다.")
                    break  # 스탑 조건
                    ###여기는 리뷰를 보다보면 리뷰 내용 없이 별점만 있는 곳이 있습니다. 그런 곳은 필요가 없다고 판단해 리뷰가 없는 부분이 나오면 코드를 중단하는 것으로 설정했습니다.

                # CSV 파일에 저장
                with open('/content/reviews.csv', mode='a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow([PRODUCT_NAME, rating, review_content])

                print(f"별점: {rating}")
                print(f"리뷰: {review_content}")
                print()

            # 페이지가 10개 단위로 나누어져 있으므로, 1-10, 11-20, 21-30... 페이지로 넘어갑니다.
            if page_num % 10 == 1:
                # "다음 페이지" 버튼을 클릭하여 다음 페이지로 넘어가기
                try:
                    next_page_btn = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, ".sdp-review__article__page__next.sdp-review__article__page__next--active.js_reviewArticlePageNextBtn"))
                    )
                    next_page_btn.click()
                    time.sleep(random.uniform(2, 6))
                except:
                    print("다음 페이지 버튼을 클릭할 수 없습니다. 종료합니다.")
                    break  # 종료 조건

            # 페이지 번호가 바뀌면 해당 페이지로 이동
            try:
                page_btn = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, f".js_reviewArticlePageBtn[data-page='{page_num}']"))
                )
                page_btn.click()
                time.sleep(random.uniform(2, 6))
            except:
                print(f"페이지 {page_num}로 이동할 수 없습니다. 종료합니다.")
                break  # 종료 조건

            # 다음 페이지로 이동할 때마다 페이지 번호 증가
            page_num += 1

        # 다운로드 하기
        print("크롤링 완료! CSV 파일을 다운로드 중...")
        files.download('/content/reviews.csv')  # Colab에서 파일 다운로드

if __name__ == '__main__':
    main()


Connecting to Scraping Browser...


/usr/local/lib/python3.10/dist-packages/selenium/webdriver/remote/remote_connection.py:418: UserWarning: Embedding username and password in URL could be insecure, use ClientConfig instead
  headers = self.get_remote_connection_headers(parsed_url, self._client_config.keep_alive)


Connected! Navigating...


WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: unhandled inspector error: {"code":"delayed_close","message":"Requested URL (https://ads-partners.coupang.com/.well-known/interest-group/permissions/?origin=https%3A%2F%2Fwww.coupang.com) is restricted in accordance with robots.txt. Ask your account manager to get full access for targeting this site (delayed_close)"}
  (Session info: chrome=131.0.6778.86)
